In [ ]:
from __future__ import division
import numpy 
from matplotlib import pyplot
import matplotlib.pyplot as plt
import pandas as pd
import pycwt as wavelet
from pycwt.helpers import find

In [ ]:
data = pd.read_excel('quakes.xlsx') 
print (data.head()) 
data.columns = ['Deep','Deep_Earth','Thermal','Shallow','Impact'] 

In [ ]:
dat = data['Deep']
#dat = data['Deep_Earth']
#dat = data['Thermal']
#dat = data['Shallow']
#dat = data['Impact']

title = 'Number of quakes per day'
label = 'Number of quakes'
units = 'Unit'

t0 = 1970.0  # First year of record
dt = 1/(365.25)  # In years
N = dat.size
t = numpy.arange(0,N) * dt + t0

In [ ]:
mother = wavelet.Morlet(6)
s0 = 1 * dt 
dj = 1 / 12  
J = 15 / dj  
alpha, _, _ = wavelet.ar1(dat)

In [ ]:
p = numpy.polyfit(t - t0, dat, 1)
dat_notrend = dat - numpy.polyval(p, t - t0)
std = dat_notrend.std()  
var = std ** 2  
dat_norm = dat_notrend / std 
wave, scales, freqs, coi, fft, fftfreqs = wavelet.cwt(dat_norm, dt, dj, s0, J, mother)

In [ ]:
power = (numpy.abs(wave)) ** 2
fft_power = numpy.abs(fft) ** 2
period = (1 / freqs) 
power /= scales[:, None]

In [ ]:
signif, fft_theor = wavelet.significance(1.0, dt, scales, 0, alpha,
                                         significance_level=0.95,
                                         wavelet=mother)
sig95 = numpy.ones([1, N]) * signif[:, None]
sig95 = power / sig95

In [ ]:
#figure
pyplot.close('all')
pyplot.ioff()
figprops = dict(figsize=(18, 20), dpi=300)
fig = pyplot.figure(**figprops)

#original timeseries
ax = pyplot.axes([0.1, 0.75, 0.65, 0.2])
ax.plot(t, dat, 'k', linewidth=1.)
ax.set_title('a) {}'.format(title))
ax.set_ylabel(r'{} [{}]'.format(label, units))

#normalized wavelet power spectrum
bx = pyplot.axes([0., 0.37, 0.65, 0.28], sharex=ax)
levels = [0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512]
bx.contourf(t, numpy.log2(period), numpy.log2(power), numpy.log2(levels),
            extend='both', cmap=pyplot.cm.viridis)
extent = [t.min(), t.max(), 0, max(period)]
bx.contour(t, numpy.log2(period), sig95, [-99, 1], colors='k', linewidths=2,
           extent=extent)
bx.fill(numpy.concatenate([t, t[-1:] + dt, t[-1:] + dt,
                           t[:1] - dt, t[:1] - dt]),
        numpy.concatenate([numpy.log2(coi), [1e-9], numpy.log2(period[-1:]),
                           numpy.log2(period[-1:]), [1e-9]]),
        'k', alpha=0.3, hatch='x')
bx.set_title('b) {} Wavelet Power Spectrum ({})'.format(label, mother.name))
bx.set_ylabel('Period (years)')
Yticks = 2 ** numpy.arange(numpy.ceil(numpy.log2(period.min())),
                           numpy.ceil(numpy.log2(period.max())))
bx.set_yticks(numpy.log2(Yticks))
bx.set_yticklabels(Yticks)


pyplot.show()